In [94]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

In [95]:
sentence_file="final_ctg_sent_dataset"
all_words_file="all_words_with_ipa_with_ctg"
zilla_name="Chittagong"

In [96]:
word_column=zilla_name
ipa_column=zilla_name+"_ipa"

In [97]:
df_sentence = pd.read_excel(f"{sentence_file}.xlsx")
df_all_words = pd.read_excel(f"{all_words_file}.xlsx")
df_sentence.head()

,Sentences
0,অনর বুঝ ফঅলেও বুঝে।
1,নিজর বুঝ ফঅলেও বুঝে।
2,অইনত্তোবারে আইললে গরম।
3,অক্কল তেলইনর বঅরফাতা।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।"


In [99]:
df_sentence.rename(columns={"Sentences": 'Contents'}, inplace=True)
df_sentence

,Contents
0,অনর বুঝ ফঅলেও বুঝে।
1,নিজর বুঝ ফঅলেও বুঝে।
2,অইনত্তোবারে আইললে গরম।
3,অক্কল তেলইনর বঅরফাতা।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।"
...,...
1219,"হরো হাই ভরো ফানি, ফুন্দে হদ্দে আই ন জানি।"
1220,পুঁইছে হঅর চালুনিরে- তোর ফোঁন্দতকা ছ্যাদা।
1221,হুইছে হঅর চালুনিরে- তোর ফোঁন্দতকা ছ্যাদা।
1222,ওশ ঠিক মাথা খঅরাপ।


In [100]:
print(df_all_words[word_column])
print(df_all_words[ipa_column])

0        যেত্তে
1          টাইম
2           ফায়
3         এত্তে
4             এ
          ...  
16325    ছ্যাদা
16326     হুইছে
16327        ওশ
16328     খঅরাপ
16329      হোঁশ
Name: Chittagong, Length: 16330, dtype: object
0          ɟet̪t̪e
1           ta͡i̯m
2             pʰaʲ
3           æt̪t̪e
4                e
           ...    
16325       cʰæd̪a
16326     hu͡i̯cʰe
16327           oʃ
16328    kʰɔ͡ɔ̯rap
16329         hõʃ
Name: Chittagong_ipa, Length: 16330, dtype: object


In [101]:
# if Contents of df_sentence null empty drop the row
df_sentence = df_sentence.dropna(subset=["Contents"])
df_sentence = df_sentence.reset_index(drop=True)
df_sentence.head()
df_sentence.shape

(1224, 1)

In [102]:
unique_char=[]
for i in df_sentence["Contents"]:
    for j in str(i):
        if j not in unique_char:
            unique_char.append(j)
unique_char.sort()
print(unique_char)


['\n', ' ', '!', "'", ',', '-', ':', ';', '?', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'ঢ়', 'য়', '৬', '\u200c']


In [107]:
special_char=[' ','\n', '!', "'", ',', '-', ':', ';', '?', '।','৬', '\u200c' ]

In [104]:
all_sentences=""
for i in df_sentence["Contents"]:
    all_sentences+=" _$_ "+str(i)
print(all_sentences)

 _$_ অনর বুঝ ফঅলেও বুঝে। _$_ নিজর বুঝ ফঅলেও বুঝে। _$_ অইনত্তোবারে আইললে গরম।  _$_ অক্কল তেলইনর বঅরফাতা। _$_ অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়। _$_ অউরু বাড়ি মধুর হাঁড়ি, তিন দিনঅর উঅদ্দি থাইলে ঝাঁড়ার বারি। _$_ অইয়্যা ধানর ফাতা হালা অইয়্যা ফোয়ার হতা ভালা। _$_ অউনরে অউন ন হঅয়, জুনিরে অউন হঅয়। _$_ অউনরে অউন ন কয়, জুনিরে অউন কয়। _$_ অতি সুন্দরীর নেক নাই _$_ অতি নেকাইনর ভাত নাই। _$_ অতি চালাকরভাত নাই। _$_ অতি চতুরর ভাত নাই। _$_ অইন হাইলে আরা আগা ফরে। _$_ অককৈ টুয়ার বাঁশ। _$_ এক্কই টুয়ার বাঁশ। _$_ একই চুঁয়ার বাঁশ। _$_ অহথা কথা অইবু মাইনসে হইলে অদারু দারু অইবু ফাডাত ফিষিলে। _$_ অহথা হথা অইবু মাইনসে হইলে অদারু দারু অইবু ফাডাত ফিষিলে। _$_ অতি চালাকর গলাত দরি। _$_ অতি ভক্তি যার, চোরর লক্কন তার। _$_ অদের হাম ন গরি, বে অদের হাম লই টানাটানি। _$_ অক্কর দান ফক্কত যায়। _$_ অ হইলে ভ বুজস। _$_ অইদে ইতে ন'ত অয়, ন'দে ইতে নকাইয়ত নো অয়। _$_ অভাগা যিকে চায়, সায়গর ফুওয়াই যায়। _$_ অযাগাততে তুলসী, অজাতত্তে রূপসী। _$_ অসতল্যয় অসৈত্যামি, সতল্লয় সৎ, চোরল্লয় চোরামি গর শঠল্লয় শঠ _$_ অ ভাই থমথম, উলুবনৎ আছে যেই ইতে

In [105]:
df_sentence["ipa"]=df_sentence["Contents"]
df_sentence.head()

# df_sentence["ipa"] null then replace with Contents
# df_sentence["IPA"].fillna(df_sentence["Contents"], inplace=True)

,Contents,ipa
0,অনর বুঝ ফঅলেও বুঝে।,অনর বুঝ ফঅলেও বুঝে।
1,নিজর বুঝ ফঅলেও বুঝে।,নিজর বুঝ ফঅলেও বুঝে।
2,অইনত্তোবারে আইললে গরম।,অইনত্তোবারে আইললে গরম।
3,অক্কল তেলইনর বঅরফাতা।,অক্কল তেলইনর বঅরফাতা।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।","অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।"


In [115]:
for (index,row) in df_all_words.iterrows():
    word=row[word_column]
    ipa=row[ipa_column]
    if pd.isnull(ipa) or (type(ipa) == list):
        continue
    for i in special_char:
        all_sentences=all_sentences.replace(f" {word}{i}",f" {ipa}{i}")
        all_sentences=all_sentences.replace(f"{i}{word} ",f"{i}{ipa} ")
        all_sentences=all_sentences.replace(f"-{word}{i}",f"-{ipa}{i}")
    

print(all_sentences)
    


 _$_ ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe। _$_ niɟɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe। _$_ ɔ͡i̯not̪t̪obare a͡i̯lle gɔrom।  _$_ ɔkkol t̪elinɔr bɔ͡o̯ropʰat̪a। _$_ æ͡i̯le n bɔcʰore ɔʲ, nɔ͡o̯͡i̯le nɔbbo͡i̯ bɔcʰore͡o̯ nɔ͡ɔ̯ʲ। _$_ ɔ͡u̯ru baɽi mod̪ʱur hãɽi, t̪in d̪inɔr u͡ɔ̯d̪d̪i t̪ʰa͡i̯le ɟʱãɽar bari। _$_ ɔ͡i̯ʲla d̪ʱanɔr pʰat̪a hala ɔ͡i̯ʲla pʰoʷar hɔt̪a bʱala। _$_ ɔ͡u̯nore ɔ͡u̯n n ho͡ɔ̯ʲ, ɟunire ɔ͡u̯n ho͡ɔ̯ʲ। _$_ ɔ͡u̯nore ɔ͡u̯n n kɔʲ, ɟunire ɔ͡u̯n kɔʲ। _$_ ot̪i ʃund̪orir nek na͡i̯ _$_ ot̪i neka͡i̯nɔr bʱat̪ na͡i̯। _$_ ot̪i calakorbʱat̪ na͡i̯। _$_ ot̪i cot̪uror bʱat̪ na͡i̯। _$_ ɔ͡i̯n ha͡i̯le ara aga pʰɔre। _$_ ɔkko͡i̯ tuʲar bãʃ। _$_ ækko͡i̯ tuʲar bãʃ। _$_ æko͡i̯ cũʲar bãʃ। _$_ ɔhot̪ʰa kɔt̪ʰa ɔ͡i̯bu ma͡i̯nʃe ho͡i̯le ɔd̪aru d̪aru ɔ͡i̯bu pʰadat̪ pʰiʃile। _$_ ɔhot̪ʰa hɔt̪ʰa ɔ͡i̯bu ma͡i̯nʃe ho͡i̯le ɔd̪aru d̪aru ɔ͡i̯bu pʰadat̪ pʰiʃile। _$_ ot̪i calakor gɔlat̪ d̪ori। _$_ ot̪i bʱokt̪i ɟar, coror lokkon t̪ar। _$_ ɔd̪er ham n gori, be ɔd̪er ham lɔ͡i̯ tanatani। _$_ ɔkkor d̪an pʰokkot̪o ɟaʲ। _$_ ɔ ho͡i̯le bʱɔ buɟɔʃ। _$

In [116]:
ipa_list=all_sentences.split(" _$_ ")
ipa_list.pop(0)
print(len(ipa_list))
print(df_sentence.shape)
print(ipa_list[0])
print(df_sentence["Contents"][0])
print(df_sentence["Contents"])
print(ipa_list[-2])
print(ipa_list[-1])

1224
(1224, 2)
ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
অনর বুঝ ফঅলেও বুঝে।
0                              অনর বুঝ ফঅলেও বুঝে।
1                             নিজর বুঝ ফঅলেও বুঝে।
2                          অইনত্তোবারে আইললে গরম। 
3                            অক্কল তেলইনর বঅরফাতা।
4           অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।
                           ...                    
1219     হরো হাই ভরো ফানি, ফুন্দে হদ্দে আই ন জানি।
1220    পুঁইছে হঅর চালুনিরে- তোর ফোঁন্দতকা ছ্যাদা।
1221     হুইছে হঅর চালুনিরে- তোর ফোঁন্দতকা ছ্যাদা।
1222                            ওশ ঠিক মাথা খঅরাপ।
1223                          হোঁশ ঠিক মাথা খঅরাপ।
Name: Contents, Length: 1224, dtype: object
oʃ tʰik mat̪ʰa kʰɔ͡ɔ̯rap।
hõʃ tʰik mat̪ʰa kʰɔ͡ɔ̯rap।


In [25]:
# df_sentence["new_ipa"]=ipa_list
# df_sentence.head()
# df_sentence["ipa"].fillna(df_sentence["new_ipa"], inplace=True)


In [117]:
df_sentence["ipa"]=ipa_list
df_sentence.head()

,Contents,ipa
0,অনর বুঝ ফঅলেও বুঝে।,ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
1,নিজর বুঝ ফঅলেও বুঝে।,niɟɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
2,অইনত্তোবারে আইললে গরম।,ɔ͡i̯not̪t̪obare a͡i̯lle gɔrom।
3,অক্কল তেলইনর বঅরফাতা।,ɔkkol t̪elinɔr bɔ͡o̯ropʰat̪a।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।","æ͡i̯le n bɔcʰore ɔʲ, nɔ͡o̯͡i̯le nɔbbo͡i̯ bɔcʰo..."


In [118]:
df_sentence.to_excel(f"{sentence_file}_ipa_with_bn.xlsx",index=False)

In [119]:
bangla_char_list=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১',"২", '৩', '৪', '৫', '৬', '৭', '৮', '৯']
def is_bangla(text):
  for i in text:
    if i in bangla_char_list:
      print(text)
      return True
  return False

In [120]:
def count_error(ipa_column):
  error=[]
  for ipa in ipa_column:
    if is_bangla(ipa):
      error.append(ipa)
  print(len(error))
count_error(df_sentence["ipa"])

অসতী-e ʃot̪i nind̪e, ɟuni poge manik nind̪e।
ale n-আঁডইন্যা bɔld̪, ku͡i̯ɟɟa kʰa͡i̯bar ɟɔm।
accʰa hõɟa lɔ͡i̯ a͡i̯le, kudumɔr নান lage ha͡i̯lle at̪e a͡i̯le kudum , mu͡i̯ccer নান lage।
acʰe goru, n-চয় hal
ãr c'ইয়ারবাল।
aɟɟer co͡i̯lo͡o̯ gælo, cʰɔ'অলঅ ga͡i̯n n-ɔ͡i̯l।
 al n-উইটতে bak d̪e।
at̪ɔr pʰerod̪i pʰani n-ফড়ন।
at̪ɔr erod̪ipʰani n-ফড়ন।
ala-বোলা ɟa͡i̯lle d̪ora।
iɟɟɔt̪ɔr ku͡u̯ri, ago pare cʰɔ'কুরি। 
iɟɟɔt̪ɔr ku͡u̯ri, ago pare cʰɔ'কুরি। 
iʲalre bʱaŋʲa beɽa n-দেখাইছ।
æk hɔt̪ʰaʲɔ pondit̪i n-ɟaʲ, æk ঝরে-ɔ bariʃ n-ɟaʲ।
hoʷalɔr d̪oʃe bʱat̪ নৃ-mile bʱidare ga͡i̯ld̪e ra͡i̯ʃ poʷa͡i̯le।
koʷalɔr d̪oʃe bʱat̪ নৃ-mile bʱidare ga͡i̯ld̪e ra͡i̯ʃ poʷa͡i̯le।
baɟ̃lɔr pʰede gʱi bʱat̪ n-জারে।
kut̪t̪or pʰede gʱi bʱat̪ n-জারে।
baɟ̃lɔr pʰede gʱi bʱat̪ n-জারে।
ku͡i̯rɔr pʰede gʱi bʱat̪ n-জারে।
ki gorib pʰut̪e han বেআরি-lɔge hut̪e।
ki gorib pʰut̪e han বেআরি-lɔge pʰũt̪e।
ku͡i̯ccare bɔr co͡u̯go n-দেখাইস।
kʰa͡i̯t̪o pa͡i̯le pʰo͡i̯ra bʱala kʰa͡i̯t̪o n-পাইললে pʰo͡i̯ra hala।
kʰa͡i̯t̪o n-ɟa͡i̯le more, bo͡i̯t̪o n-জাইনলে l

In [121]:
df=pd.read_excel(f"{sentence_file}_ipa_with_bn.xlsx")
df=df.rename(columns={"ipa": "ipa"})
df.head()
all_ipa=[]
for ipa in df["ipa"]:
  all_ipa.append(str(ipa))
df["ipa"]=all_ipa
count_error(df["ipa"])
df.head()

অসতী-e ʃot̪i nind̪e, ɟuni poge manik nind̪e।
ale n-আঁডইন্যা bɔld̪, ku͡i̯ɟɟa kʰa͡i̯bar ɟɔm।
accʰa hõɟa lɔ͡i̯ a͡i̯le, kudumɔr নান lage ha͡i̯lle at̪e a͡i̯le kudum , mu͡i̯ccer নান lage।
acʰe goru, n-চয় hal
ãr c'ইয়ারবাল।
aɟɟer co͡i̯lo͡o̯ gælo, cʰɔ'অলঅ ga͡i̯n n-ɔ͡i̯l।
 al n-উইটতে bak d̪e।
at̪ɔr pʰerod̪i pʰani n-ফড়ন।
at̪ɔr erod̪ipʰani n-ফড়ন।
ala-বোলা ɟa͡i̯lle d̪ora।
iɟɟɔt̪ɔr ku͡u̯ri, ago pare cʰɔ'কুরি। 
iɟɟɔt̪ɔr ku͡u̯ri, ago pare cʰɔ'কুরি। 
iʲalre bʱaŋʲa beɽa n-দেখাইছ।
æk hɔt̪ʰaʲɔ pondit̪i n-ɟaʲ, æk ঝরে-ɔ bariʃ n-ɟaʲ।
hoʷalɔr d̪oʃe bʱat̪ নৃ-mile bʱidare ga͡i̯ld̪e ra͡i̯ʃ poʷa͡i̯le।
koʷalɔr d̪oʃe bʱat̪ নৃ-mile bʱidare ga͡i̯ld̪e ra͡i̯ʃ poʷa͡i̯le।
baɟ̃lɔr pʰede gʱi bʱat̪ n-জারে।
kut̪t̪or pʰede gʱi bʱat̪ n-জারে।
baɟ̃lɔr pʰede gʱi bʱat̪ n-জারে।
ku͡i̯rɔr pʰede gʱi bʱat̪ n-জারে।
ki gorib pʰut̪e han বেআরি-lɔge hut̪e।
ki gorib pʰut̪e han বেআরি-lɔge pʰũt̪e।
ku͡i̯ccare bɔr co͡u̯go n-দেখাইস।
kʰa͡i̯t̪o pa͡i̯le pʰo͡i̯ra bʱala kʰa͡i̯t̪o n-পাইললে pʰo͡i̯ra hala।
kʰa͡i̯t̪o n-ɟa͡i̯le more, bo͡i̯t̪o n-জাইনলে l

,Contents,ipa
0,অনর বুঝ ফঅলেও বুঝে।,ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
1,নিজর বুঝ ফঅলেও বুঝে।,niɟɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
2,অইনত্তোবারে আইললে গরম।,ɔ͡i̯not̪t̪obare a͡i̯lle gɔrom।
3,অক্কল তেলইনর বঅরফাতা।,ɔkkol t̪elinɔr bɔ͡o̯ropʰat̪a।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।","æ͡i̯le n bɔcʰore ɔʲ, nɔ͡o̯͡i̯le nɔbbo͡i̯ bɔcʰo..."


In [122]:
print(df.shape)

(1224, 2)


In [123]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    # print(f"Started {row.Index}")
    if not is_bangla(row.ipa):
      continue
    print(f"Started {row.Index} {row.Contents}")
    words=row.ipa.split(" ")
    if(len(words)>1):
      datas=""
      for word in words:
        # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
        if not is_bangla(word):
          datas=datas+" "+word
          continue
        url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            datas=datas+" "+data["data"][0]
          else:
            datas=datas+" "+"ERROR"
        except:
          print(f"error at{row.Index}:{row.Contents}")
          datas=datas+" "+"ERROR"
      print(f"Ended {row.Index}")
      df.at[row.Index,"ipa"]=datas
    else:
      url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.Contents}"
      response=None
      try:
        response = requests.get (url, verify=False)
        if response and response.status_code == 200:
          data= response.json()
          df.at[row.Index,"ipa"]=data["data"][0]
          print(f"Ended {row.Index}:{row.Contents}")
          print(f"{row.Index}:{data['data'][0]}")
        else:
          data= None
      except Exception as e:
        print(e)
        print(f"error at{row.Index}:{row.Contents}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [124]:

start=00
steps=1000
end=df.shape[0]
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_words, subset))
print(pool.shutdown())
df.to_csv(f"{sentence_file}_{start_pos}_{end}.csv",index=False)

                                           Contents  \
0                               অনর বুঝ ফঅলেও বুঝে।   
1                              নিজর বুঝ ফঅলেও বুঝে।   
2                           অইনত্তোবারে আইললে গরম।    
3                             অক্কল তেলইনর বঅরফাতা।   
4            অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।   
..                                              ...   
995                                   মাইল্লা পিরা।   
996          মইষর ব্যাঁয়া শিং গুতা মাইরতার সমত ওজু।   
997                   মও ভাইনা যিঅত হকক্ল কাম হিঅত।   
998              মা চাইতে ঝি চ'অ বাপ চাইতে পুত চ'অ।   
999  মাইনষর আদর আইয়নে যঅনে গরু ছলর আদর লেঅনে ফোছনে।   

                                                   ipa  
0                         ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।  
1                        niɟɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।  
2                      ɔ͡i̯not̪t̪obare a͡i̯lle gɔrom।   
3                        ɔkkol t̪elinɔr bɔ͡o̯ropʰat̪a।  
4    æ͡i̯le n bɔcʰore ɔʲ, nɔ͡o̯͡i̯le nɔbbo͡i̯ bɔcʰo...

In [125]:
count_error(df["ipa"])

0


In [46]:
df.drop(columns=["new_ipa"],inplace=True)
df.head()

,Index,District,External_ID,Contents,ipa
0,0,Tangail,rec_40_audio_27.wav,যদিও এতো বড় আগুন লাগে তাইলে সেইটা নিবাইতে হবে না?,ɟod̪ɪ͡o̯ et̪o bɔɽo ɐgun lɐge t̪ɐ͡ɪ̯le ʃe͡ɪ̯tɐ ...
1,1,Tangail,rec_40_audio_27.wav,আর সব থেকে ফানি কথা হইতেছে দেখ?,ɐɾ ʃɔb t̪ʰeke pʰɐnɪ kɔt̪ʰɐ ho͡ɪ̯t̪ecʰe d̪ɛkʰo?
2,2,Tangail,rec_40_audio_27.wav,আগুন লাগছে আমাদের মাননীয় প্রধানমন্ত্রী সে <> অ...,ɐgun lɐgcʰe ɐmɐd̪eɾ mɐnonɪʲo pɾod̪ʱɐnmont̪ɾɪ ʃ...
3,3,Tangail,rec_40_audio_27.wav,সেইটা নিয়া তার মাথাব্যাথা হইয়া গেছে।,ʃe͡ɪ̯tɐ nɪʲɐ t̪ɐɾ mɐt̪ʰɐbɛt̪ʰɐ ho͡ɪ̯ʲɐ gecʰe।
4,4,Tangail,rec_40_audio_27.wav,"হুম, এইটা এইটার বিরুদ্ধে স্টেপ নিবে।","hum, e͡ɪ̯tɐ e͡ɪ̯tɐɾ bɪɾud̪d̪ʱe step nɪbe।"


In [127]:
df.to_excel(f"{sentence_file}_final.xlsx",index=False)

In [126]:
df.head()

,Contents,ipa
0,অনর বুঝ ফঅলেও বুঝে।,ɔnɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
1,নিজর বুঝ ফঅলেও বুঝে।,niɟɔr buɟʱ pʰɔ͡o̯le͡o̯ buɟʱe।
2,অইনত্তোবারে আইললে গরম।,ɔ͡i̯not̪t̪obare a͡i̯lle gɔrom।
3,অক্কল তেলইনর বঅরফাতা।,ɔkkol t̪elinɔr bɔ͡o̯ropʰat̪a।
4,"অইলে ন বছরে অয়, নঅইলে নব্বই বছরেও নঅয়।","æ͡i̯le n bɔcʰore ɔʲ, nɔ͡o̯͡i̯le nɔbbo͡i̯ bɔcʰo..."


# Done


In [85]:
df=pd.read_excel("ctg_extra_words_withIPA.xlsx")
df_all_words=pd.read_excel("all_words_with_ipa.xlsx")

In [86]:
df

,words,ipa
0,অনর,ɔnɔr
1,নিজর,niɟɔr
2,বুঝ,buɟʱ
3,ফঅলেও,pʰɔ͡o̯le͡o̯
4,বুঝে,buɟʱe
...,...,...
3671,ছ্যাদা,cʰæd̪a
3672,হুইছে,hu͡i̯cʰe
3673,ওশ,oʃ
3674,খঅরাপ,kʰɔ͡ɔ̯rap


In [87]:
len(df_all_words["Chittagong"])

12654

In [88]:
# concat rows of df_all_words and df on word_column

# df_all_words["Chittagong"]=df_all_words["Chittagong"].append(df["words"],ignore_index=True)
df=df.rename(columns={"words": "Chittagong","ipa":"Chittagong_ipa"})
df.shape





(3676, 2)

In [90]:
df_all_words


,Rangpur,Rangpur_ipa,Kishoreganj,Kishoreganj_ipa,Chittagong,Chittagong_ipa,Narsingdi,Narsingdi_ipa,Narail,Narail_ipa,Tangail,Tangail_ipa
0,লেখিয়্যা,lekʰiʲt̪a,আটাইশ,ata͡i̯ʃ,যেত্তে,ɟet̪t̪e,তারা,t̪ara,আমি,ami,যদিও,ɟod̪i͡o̯
1,কালি,kali,করছিলাম,korcʰilam,টাইম,ta͡i̯m,তো,t̪o,তো,t̪o,এতো,et̪o
2,দিয়্যা,d̪ikt̪a,দের,d̪er,ফায়,pʰaʲ,ওই,o͡i̯,চইলে,co͡i̯le,বড়,bɔɽo
3,সেই,ʃe͡i̯,কানি,kani,এত্তে,æt̪t̪e,খারাপ,kʰarap,আসলাম,aʃlam,আগুন,agun
4,সুন্দর,ʃund̪ɔr,ক্ষেত,kʰet̪,এ,e,খেইলাই,kʰe͡i̯la͡i̯,তুমারে,t̪umare,লাগে,lage
...,...,...,...,...,...,...,...,...,...,...,...,...
12649,NaN,NaN,তাকলাম,t̪aklam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12650,NaN,NaN,হোয়াটসআপই,hoʷatʃapi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12651,NaN,NaN,হোয়াটসআপ,hoʷatʃɔ͡a̯p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12652,NaN,NaN,বউয়েরতে,bo͡u̯ʲert̪e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_new=pd.concat([df_all_words,df],axis=0,ignore_index=True,keys=["Chittagong","Chittagong_ipa"])
df_new

,Rangpur,Rangpur_ipa,Kishoreganj,Kishoreganj_ipa,Chittagong,Chittagong_ipa,Narsingdi,Narsingdi_ipa,Narail,Narail_ipa,Tangail,Tangail_ipa
0,লেখিয়্যা,lekʰiʲt̪a,আটাইশ,ata͡i̯ʃ,যেত্তে,ɟet̪t̪e,তারা,t̪ara,আমি,ami,যদিও,ɟod̪i͡o̯
1,কালি,kali,করছিলাম,korcʰilam,টাইম,ta͡i̯m,তো,t̪o,তো,t̪o,এতো,et̪o
2,দিয়্যা,d̪ikt̪a,দের,d̪er,ফায়,pʰaʲ,ওই,o͡i̯,চইলে,co͡i̯le,বড়,bɔɽo
3,সেই,ʃe͡i̯,কানি,kani,এত্তে,æt̪t̪e,খারাপ,kʰarap,আসলাম,aʃlam,আগুন,agun
4,সুন্দর,ʃund̪ɔr,ক্ষেত,kʰet̪,এ,e,খেইলাই,kʰe͡i̯la͡i̯,তুমারে,t̪umare,লাগে,lage
...,...,...,...,...,...,...,...,...,...,...,...,...
16325,NaN,NaN,NaN,NaN,ছ্যাদা,cʰæd̪a,NaN,NaN,NaN,NaN,NaN,NaN
16326,NaN,NaN,NaN,NaN,হুইছে,hu͡i̯cʰe,NaN,NaN,NaN,NaN,NaN,NaN
16327,NaN,NaN,NaN,NaN,ওশ,oʃ,NaN,NaN,NaN,NaN,NaN,NaN
16328,NaN,NaN,NaN,NaN,খঅরাপ,kʰɔ͡ɔ̯rap,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
len(df_new["Chittagong"])

16330

In [93]:
df_new.to_excel("all_words_with_ipa_with_ctg.xlsx",index=False)